# Fetching Tweets for Training Data from Twitter API

## Twitter API Premium v1.1: Search Tweets 30-Day Endpoint

In [227]:
# Import dependencies
import tweepy
import pandas as pd
import json

In [228]:
# Import API client token
import os
from dotenv import load_dotenv
load_dotenv()

twit_token = os.getenv('twit_token')

In [229]:
auth = tweepy.OAuth2BearerHandler(twit_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [230]:
# Code reproduced from Twitter developer API documentation. Additional parsing features and customizations made where necessary for desired output.
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type

parsedTweets = []
rawTweets = []

def parse_tweets(status):
    for tweet in status:
        
        rawTweets.append(tweet._json)
        hashtags = []
        mentions = []
        
        if determine_tweet_type(tweet._json) == 'Retweet':
            if 'extended_tweet' in tweet._json['retweeted_status']:
                full_text = tweet._json['retweeted_status']['extended_tweet']['full_text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json['retweeted_status']["extended_tweet"]["entities"]["hashtags"] 
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json['retweeted_status']["extended_tweet"]["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
                        
            else:
                full_text = tweet._json['retweeted_status']['text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json['retweeted_status']["entities"]["hashtags"]
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json['retweeted_status']["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
               

        elif determine_tweet_type(tweet._json) == 'Quote Tweet':
            if 'extended_tweet' in tweet._json['quoted_status']:
                full_text = tweet._json['quoted_status']['extended_tweet']['full_text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json['quoted_status']["extended_tweet"]["entities"]["hashtags"]
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json['quoted_status']["extended_tweet"]["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
                    
            else:
                full_text = tweet._json['quoted_status']['text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json['quoted_status']["entities"]["hashtags"]
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json["quoted status"]["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
                
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json["extended_tweet"]["entities"]["hashtags"]
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json["extended_tweet"]["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
                    
            else:
                full_text = tweet._json['text']
                
                # Collect hashtags from tweet into list
                hashtags_entity = tweet._json["entities"]["hashtags"]
                if len(hashtags_entity) == 0:
                    hashtags = None
                else:
                    for hashtag in hashtags_entity:
                        hashtags.append("#" + hashtag['text'])
                
                # Collect mentions from tweet into list
                mentions_entity = tweet._json["entities"]["user_mentions"]
                if len(mentions_entity) == 0:
                    mentions = None
                else:
                    for mention in mentions_entity:
                        mentions.append("@" + mention["screen_name"])
        
                
        # Get coordinates latitude and longitude into separate variables by subscripting coordinates if coordinates object not Nonetype
        coordinates = tweet._json["coordinates"]
        if coordinates is not None:
            coord_lat = tweet._json["coordinates"]["coordinates"][0]
            coord_lng = tweet._json["coordinates"]["coordinates"][1]
        else:
            coord_lat = None
            coord_lng = None
            
        
        # Get place attributes into separate variables by subscripting place if place object not Nonetype 
        place = tweet._json["place"]
        if place is not None:
            place_type = tweet._json["place"]["place_type"]
            place_name = tweet._json["place"]["name"]
            place_full_name = tweet._json["place"]["full_name"]
            country_code = tweet._json["place"]["country_code"]
            country = tweet._json["place"]["country"]
        else:
            place_type = None
            place_name = None
            place_full_name = None
            country_code = None
            country = None
            
        # Parsing profile geo data

        # First intialize profile geo field values as Nonetype for the default value (value if field not found in profile geo data)
        prof_country = None
        prof_country_code = None
        prof_locality = None
        prof_region = None
        prof_sub_region = None
        prof_full_name = None
        prof_coord_lat = None
        prof_coord_lng = None 

        for field in tweet._json["user"]: 
            if field == "derived":
                prof_geo_data = tweet._json["user"]["derived"]
                if prof_geo_data["locations"] is not None:
                    for field in prof_geo_data["locations"][0]:
                        if field == "country":
                            prof_country = prof_geo_data["locations"][0][field]

                        elif field == "country_code":
                            prof_country_code = prof_geo_data["locations"][0][field]

                        elif field == "locality":
                            prof_locality = prof_geo_data["locations"][0][field]

                        elif field == "region":
                            prof_region = prof_geo_data["locations"][0][field]

                        elif field == "sub_region":
                            prof_sub_region = prof_geo_data["locations"][0][field]

                        elif field == "full_name":
                            prof_full_name = prof_geo_data["locations"][0][field]

                        elif field == "geo":
                            prof_coord_lat = prof_geo_data["locations"][0][field]["coordinates"][0]
                            prof_coord_lng = prof_geo_data["locations"][0][field]["coordinates"][1]

                        else:
                            pass
                else:
                    pass
            else:
                pass
            

            
        
        mydict = { "tweet_id": tweet._json["id_str"], #Tweet ID
                       "date":tweet._json["created_at"], #Timestamp of tweet creation
                       "full_text": full_text, #Full tweet text
                       "tweet_type": determine_tweet_type(tweet._json), #Type of tweet
                       "hashtags": hashtags, #List of hashtags used in the tweet
                       "mentions": mentions, #List of mentions used in the tweet
                       "user_location": tweet._json["user"]["location"], #The user's listed location
                       "geo": tweet._json["geo"], #Geodata 'geo' attribute
                       "lat_coordinates": coord_lat, #Geodata 'latitude coordinates' from 'coordinates' attribute
                       "lng_coordinates": coord_lng, #Geodata 'longitude coordinates' from 'coordinates' attribute
                       "place_type": place_type, #Geodata 'place_type' from 'place' attribute
                       "place_name": place_name, #Geodata 'name' from 'place' attribute
                       "place_full_name": place_full_name, #Geodata 'full_name' from 'place' attribute
                       "country_code": country_code, #Geodata 'country_code' from 'place' attribute
                       "country": country, #Geodata 'country' from 'place' attribute
                       "prof_country": prof_country, #Profile geodata 'country' from 'user''derived''locations'
                       "prof_country_code": prof_country_code, #Profile geodata 'country_code' from 'user''derived''locations'
                       "prof_locality": prof_locality, #Profile geodata 'locality' from 'user''derived''locations'
                       "prof_region": prof_region, #Profile geodata 'region' from 'user''derived''locations'
                       "prof_sub_region": prof_sub_region, #Profile geodata 'sub_region' from 'user''derived''locations'
                       "prof_full_name": prof_full_name, #Profile geodata 'full_name' from 'user''derived''locations'
                       "prof_coord_lat": prof_coord_lat, #Profile geodata 'latitude' from 'user''derived''locations''geo''coordinates'
                       "prof_coord_lng": prof_coord_lng, #Profile geodata 'longitude' from 'user''derived''locations''geo''coordinates'
                       "reply_count": tweet._json["reply_count"], #Number of times Tweet has been replied to
                       "quote_count": tweet._json["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet._json["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet._json["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"] #Link to tweet
              }
        
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list

In [231]:
# Code to run query against the Twitter API v1.1 Premium - 30 day search endpoint for fetching tweets in the last 30-day archive 

query = '(#guncontrol OR #gunrights OR #gunsense OR #2a OR #gunviolence OR #nra OR #guns OR #shooting OR #firearms OR #gunsafety OR #uvalde \
OR "gun control" OR "2nd amendment" OR ("gun" "violence") OR "right to bear" OR "nra" OR ("gun" "safety") OR ("gun" "rights") \
OR "good guy with a gun" OR "assault weapons" OR ("ban" "guns") OR "ban assault" OR ("school" "shootings") OR "ar-15" OR "mass shootings" \
OR "open carry" OR "gun owners" \ OR "concealed carry" OR "gun laws" OR "gun law" OR "pro-gun" OR "anti-gun" OR "gun safety" OR "high capacity magazine" \
OR "second amendment" OR "gun lobby" OR "gun ownership" OR "carry permit" OR "bump stock" OR "gunman" OR "red flag laws" OR "red-flag laws" OR "brady law" \
OR "gun sales" OR ("buying" ("gun" OR "guns")) OR "gun shows" OR "gun sense" OR "sandy hook" OR ("guns" ("republicans" OR "democrats")) OR (("background check" OR "background checks") \
("gun" OR "guns" OR "sale" OR "sales")) OR (("legal" OR "illegal") ("gun" OR "guns"))) lang:en profile_country:US -is:retweet -is:quote'


# Initalize lists to populate with raw and parsed using parsing function
rawTweets = []
parsedTweets = []

# Fetch tweets from the endpoint using pagination to collect up to 10,000 tweets for the specified day (UTC)
status = tweepy.Cursor(api.search_30_day, label='gunsentiment', query=query, fromDate='202211031100', toDate='202211040000', maxResults='500').items(500)

# parse the fetched tweets and store key-value pairs in list of dictionaries in parsedTweets. Append raw JSON responses to rawTweets list
parse_tweets(status)


# convert raw tweets JSON responses array into .json file
with open("../res/training_tweets_raw.json", "w") as training_tweets:
    json.dump(rawTweets, training_tweets)
    
    
tweet_df = pd.DataFrame(parsedTweets)

In [232]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tweet_id           500 non-null    object 
 1   date               500 non-null    object 
 2   full_text          500 non-null    object 
 3   tweet_type         500 non-null    object 
 4   hashtags           61 non-null     object 
 5   mentions           423 non-null    object 
 6   user_location      500 non-null    object 
 7   geo                0 non-null      object 
 8   lat_coordinates    0 non-null      object 
 9   lng_coordinates    0 non-null      object 
 10  place_type         8 non-null      object 
 11  place_name         8 non-null      object 
 12  place_full_name    8 non-null      object 
 13  country_code       8 non-null      object 
 14  country            8 non-null      object 
 15  prof_country       499 non-null    object 
 16  prof_country_code  499 non

In [233]:
tweet_df.head(10)

,tweet_id,date,full_text,tweet_type,hashtags,mentions,user_location,geo,lat_coordinates,lng_coordinates,...,prof_region,prof_sub_region,prof_full_name,prof_coord_lat,prof_coord_lng,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1588320083335467009,Thu Nov 03 23:59:52 +0000 2022,@UnnecRoughness A is Levis. \nB is KJ Jefferso...,Reply Tweet,None,[@UnnecRoughness],"Arkansas, USA",None,None,None,...,Arkansas,None,"Arkansas, United States",-92.50044,34.75037,0,0,6,0,https://twitter.com/twitter/status/15883200833...
1,1588320019259469824,Thu Nov 03 23:59:36 +0000 2022,Republican candidate's kids are almost KILLED ...,Original Tweet,None,None,United States,None,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883200192...
2,1588320006840160256,Thu Nov 03 23:59:33 +0000 2022,@Jupiter62214807 @cjstheman_611 @BMC_MacDaddy ...,Reply Tweet,None,"[@Jupiter62214807, @cjstheman_611, @BMC_MacDad...","Virginia, USA",None,None,None,...,West Virginia,None,"West Virginia, United States",-80.50009,38.50038,0,0,1,0,https://twitter.com/twitter/status/15883200068...
3,1588319891446272001,Thu Nov 03 23:59:06 +0000 2022,"@davidhogg111 It’s 2022, get new talking point...",Reply Tweet,None,[@davidhogg111],MI,None,None,None,...,Michigan,None,"Michigan, United States",-85.50033,44.25029,0,0,8,0,https://twitter.com/twitter/status/15883198914...
4,1588319866385477632,Thu Nov 03 23:59:00 +0000 2022,Watch: Gunman opens fire on Imran Khan https:/...,Original Tweet,None,None,USA,None,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883198663...
5,1588319827651104769,Thu Nov 03 23:58:51 +0000 2022,@Stormof69 @VoteYesOn114 I’m always amused tha...,Reply Tweet,None,"[@Stormof69, @VoteYesOn114]","Manchester, CT",None,None,None,...,Connecticut,Hartford County,"Manchester, Connecticut, United States",-72.52148,41.77593,0,0,4,0,https://twitter.com/twitter/status/15883198276...
6,1588319810181529602,Thu Nov 03 23:58:46 +0000 2022,@ColinCowherd @DrHoodieMaddog What about kids ...,Reply Tweet,None,"[@ColinCowherd, @DrHoodieMaddog]","Brooklyn, NY",None,None,None,...,New York,Kings County,"Brooklyn, New York, United States",-73.94958,40.65010,0,0,0,0,https://twitter.com/twitter/status/15883198101...
7,1588319807375433728,Thu Nov 03 23:58:46 +0000 2022,@Unnoble_Savage @brianleeellis @michaeljknowle...,Reply Tweet,None,"[@Unnoble_Savage, @brianleeellis, @michaeljkno...","Willits, CA",None,None,None,...,California,Mendocino County,"Willits, California, United States",-123.35557,39.40961,0,0,0,0,https://twitter.com/twitter/status/15883198073...
8,1588319706343124992,Thu Nov 03 23:58:22 +0000 2022,@RobSchneider Let’s keep the 2nd Amendment and...,Reply Tweet,None,[@RobSchneider],"Monterey, CA",None,None,None,...,California,Monterey County,"Monterey, California, United States",-121.89468,36.60024,0,0,0,0,https://twitter.com/twitter/status/15883197063...
9,1588319697808019456,Thu Nov 03 23:58:20 +0000 2022,@davidhogg111 How many times a year are guns u...,Reply Tweet,None,[@davidhogg111],USA,None,None,None,...,None,None,United States,-98.50000,39.76000,0,0,0,0,https://twitter.com/twitter/status/15883196978...


In [238]:
tweet_df.loc[:, ['user_location', 'prof_country', 'prof_country_code', 'prof_region', 'prof_sub_region', 'prof_full_name']].head(10)
# add , 'prof_locality'

,user_location,prof_country,prof_country_code,prof_region,prof_sub_region,prof_full_name
0,"Arkansas, USA",United States,US,Arkansas,None,"Arkansas, United States"
1,United States,United States,US,None,None,United States
2,"Virginia, USA",United States,US,West Virginia,None,"West Virginia, United States"
3,MI,United States,US,Michigan,None,"Michigan, United States"
4,USA,United States,US,None,None,United States
5,"Manchester, CT",United States,US,Connecticut,Hartford County,"Manchester, Connecticut, United States"
6,"Brooklyn, NY",United States,US,New York,Kings County,"Brooklyn, New York, United States"
7,"Willits, CA",United States,US,California,Mendocino County,"Willits, California, United States"
8,"Monterey, CA",United States,US,California,Monterey County,"Monterey, California, United States"
9,USA,United States,US,None,None,United States


In [239]:
tweet_df['tweet_type'].value_counts()

Reply Tweet       401
Original Tweet     99
Name: tweet_type, dtype: int64

In [240]:
tweet_df['country_code'].value_counts()

US    8
Name: country_code, dtype: int64

In [241]:
import numpy as np
tweet_df.loc[tweet_df['country_code'].notnull()].T

,26,53,69,263,328,402,425,435
tweet_id,1588319191559307264,1588318339960446976,1588317740850610176,1588311668987396096,1588309708464885760,1588307007840231425,1588305891383713792,1588305347609268224
date,Thu Nov 03 23:56:19 +0000 2022,Thu Nov 03 23:52:56 +0000 2022,Thu Nov 03 23:50:33 +0000 2022,Thu Nov 03 23:26:25 +0000 2022,Thu Nov 03 23:18:38 +0000 2022,Thu Nov 03 23:07:54 +0000 2022,Thu Nov 03 23:03:28 +0000 2022,Thu Nov 03 23:01:18 +0000 2022
full_text,@davidhogg111 Um…no group does more for gun sa...,@ColinCowherd So just forget about the school ...,(6) “Families in #Uvalde till this day are cal...,@RawStory @morgfair 2nd Amendment! Repeal it i...,@ReverendWarnock @BarackObama The @GOP have vo...,Republican candidate's kids are almost KILLED ...,@brixwe His plan is to take over America witho...,@JimlikesBB @AngryFleas @katiehobbs If you don...
tweet_type,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Reply Tweet,Original Tweet,Reply Tweet,Reply Tweet
hashtags,None,None,"[#Uvalde, #UHCLASSPanos]",None,None,None,None,None
mentions,[@davidhogg111],[@ColinCowherd],None,"[@RawStory, @morgfair]","[@ReverendWarnock, @BarackObama, @GOP]",None,[@brixwe],"[@JimlikesBB, @AngryFleas, @katiehobbs]"
user_location,"Las Vegas, Nevada","Scottsdale, AZ","Houston, TX",South Park Colorado,"West Palm Beach, FL","Arlington Heights, VA","Haverhill, MA",United States
geo,None,None,None,None,None,None,None,None
lat_coordinates,None,None,None,None,None,None,None,None
lng_coordinates,None,None,None,None,None,None,None,None


In [226]:
# Parsing profile geo data code snippet

# First intialize attributes as Nonetype for the default value (value if attribute not found in profile geo data)
prof_country = None
prof_country_code = None
prof_locality = None
prof_region = None
prof_sub_region = None
prof_full_name = None
prof_coord_lat = None
prof_coord_lng = None
prof_geo_data = rawTweets[2]["user"]["derived"] 

if prof_geo_data is not None:
    if prof_geo_data["locations"] is not None:
        for field in prof_geo_data["locations"][0]:
            if field == "country":
                prof_country = prof_geo_data["locations"][0][field]
                print(prof_country)
            elif field == "country_code":
                prof_country_code = prof_geo_data["locations"][0][field]
                print(prof_country_code)
            elif field == "locality":
                prof_locality = prof_geo_data["locations"][0][field]
                print(prof_locality)
            elif field == "region":
                prof_region = prof_geo_data["locations"][0][field]
                print(prof_region)
            elif field == "sub_region":
                prof_sub_region = prof_geo_data["locations"][0][field]
                print(prof_sub_region)
            elif field == "full_name":
                prof_full_name = prof_geo_data["locations"][0][field]
                print(prof_full_name)
            elif field == "geo":
                prof_coord_lat = prof_geo_data["locations"][0][field]["coordinates"][0]
                prof_coord_lng = prof_geo_data["locations"][0][field]["coordinates"][1]
                print(prof_coord_lat)
                print(prof_coord_lng)       
            else:
                pass
    else:
        pass
else:
    pass

United States
US
Michigan
Michigan, United States
-85.50033
44.25029


In [155]:
rawTweets[2]["user"]["derived"]["locations"][0]["geo"]["coordinates"]

[-72.52148, 41.77593]

In [169]:
for mention in rawTweets[429]["extended_tweet"]["entities"]["user_mentions"]:
    print(mention["screen_name"])
for hashtag in rawTweets[429]["extended_tweet"]["entities"]["hashtags"]:
    print(hashtag["text"])
                        
        #hashtags.append("#" + hashtag['text'])
                        
    #mentions.append("@" + tweet._json['quoted_status']["extended_tweet"]["entities"]["user_mentions"][mention]["screen_name"])

coleg
copolitics
2A
gunrights


In [163]:
rawTweets[2]

{'created_at': 'Thu Nov 03 23:58:51 +0000 2022',
 'id': 1588319827651104769,
 'id_str': '1588319827651104769',
 'text': '@Stormof69 @VoteYesOn114 I’m always amused that 2A’s don’t see gun control as the ‘well regulated’ part.',
 'display_text_range': [25, 104],
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': 1588268919131090945,
 'in_reply_to_status_id_str': '1588268919131090945',
 'in_reply_to_user_id': 179040242,
 'in_reply_to_user_id_str': '179040242',
 'in_reply_to_screen_name': 'Stormof69',
 'user': {'id': 412772316,
  'id_str': '412772316',
  'name': 'tim maynard',
  'screen_name': 'darthmaynardo',
  'location': 'Manchester, CT',
  'url': 'http://www.timmaynard.com',
  'description': 'Highland Guitar Studio',
  'translator_type': 'none',
  'derived': {'locations': [{'country': 'United States',
     'country_code': 'US',
     'locality': 'Manchester',
     'region': 'Connecticut',
     

In [71]:
range(len(rawTweets[0]["user"]["derived"]["locations"][0]))

range(0, 5)

In [40]:
print(json.dumps(rawTweets[0], indent=2))

{
  "created_at": "Thu Nov 03 23:56:19 +0000 2022",
  "id": 1588319191559307264,
  "id_str": "1588319191559307264",
  "text": "@davidhogg111 Um\u2026no group does more for gun safety than the NRA.\nDo your homework \ud83d\ude0e\ud83c\udf78\ud83c\uddfa\ud83c\uddf8\ud83e\udd85",
  "display_text_range": [
    14,
    87
  ],
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
  "truncated": false,
  "in_reply_to_status_id": 1588181796662829057,
  "in_reply_to_status_id_str": "1588181796662829057",
  "in_reply_to_user_id": 1915033663,
  "in_reply_to_user_id_str": "1915033663",
  "in_reply_to_screen_name": "davidhogg111",
  "user": {
    "id": 84381404,
    "id_str": "84381404",
    "name": "Bob(Marcello)Martini",
    "screen_name": "binitram",
    "location": "Las Vegas,  Nevada",
    "url": "http://www.martinifinancial.com",
    "description": "From Western PA, now living in Las Vegas. Married with 3 children. Love to travel!! Huge Steeler

In [67]:
rawTweets[0:2]

[{'created_at': 'Thu Nov 03 23:59:52 +0000 2022',
  'id': 1588320083335467009,
  'id_str': '1588320083335467009',
  'text': '@UnnecRoughness A is Levis. \nB is KJ Jefferson. \nThe same people who think Levis is an NFL Qb is the same people wh… https://t.co/iwYZ3fp45t',
  'display_text_range': [16, 140],
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'truncated': True,
  'in_reply_to_status_id': 1588206585943318529,
  'in_reply_to_status_id_str': '1588206585943318529',
  'in_reply_to_user_id': 988517346032672779,
  'in_reply_to_user_id_str': '988517346032672779',
  'in_reply_to_screen_name': 'UnnecRoughness',
  'user': {'id': 531999121,
   'id_str': '531999121',
   'name': 'Jake Alexander',
   'screen_name': 'JakeAlexander73',
   'location': 'Arkansas, USA',
   'url': None,
   'description': 'Callin the hogs since 96! married to @victorialou_53',
   'translator_type': 'none',
   'derived': {'locations': [{'country': 'United States',
 

In [212]:
# # Export tweet_df as training data CSV
# tweet_df.to_csv('../res/training_tweets.csv', index=False)